In [12]:
import numpy as np
import cupy as cp
import time

In [13]:
def matrix_multiply_cpu(matrix_a, matrix_b):
    # Проверка корректности перемножения
    if matrix_a.shape[1] != matrix_b.shape[0]:
        raise ValueError("Неправильные размеры матриц для перемножения")

    result = np.zeros((matrix_a.shape[0], matrix_b.shape[1]))

    start_time = time.time()
    for i in range(matrix_a.shape[0]):
        for j in range(matrix_b.shape[1]):
            for k in range(matrix_a.shape[1]):
                result[i, j] += matrix_a[i, k] * matrix_b[k, j]
    end_time = time.time()

    return result, end_time - start_time

In [14]:
matrix_mult_kernel = cp.RawKernel(r'''
extern "C" __global__
void matrix_mult(const float* A, const float* B, float* C, int N, int M, int K) {
    // Calculate global thread indices
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < K) {
        float value = 0.0;
        for (int k = 0; k < M; ++k) {
            value += A[row * M + k] * B[k * K + col];
        }
        C[row * K + col] = value;
    }
}
''', 'matrix_mult')


In [15]:
def matrix_multiply_gpu(matrix_a, matrix_b):
    # Check dimensions
    if matrix_a.shape[1] != matrix_b.shape[0]:
        raise ValueError("Неправильные размеры матриц для перемножения")

    # Transfer matrices to GPU
    matrix_a_gpu = cp.asarray(matrix_a)
    matrix_b_gpu = cp.asarray(matrix_b)

    # Prepare a result matrix
    result_gpu = cp.empty((matrix_a.shape[0], matrix_b.shape[1]), dtype=cp.float32)

    # Define grid and block sizes
    N, M = matrix_a.shape
    K = matrix_b.shape[1]
    block_size = (16, 16)  # Set block size
    grid_size = (int(np.ceil(K / block_size[0])), int(np.ceil(N / block_size[1])))

    # Measure execution time
    start_time = time.time()

    # Launch the kernel
    matrix_mult_kernel(grid_size, block_size, (matrix_a_gpu, matrix_b_gpu, result_gpu, N, M, K))

    # Retrieve the result from GPU
    result = cp.asnumpy(result_gpu)

    end_time = time.time()
    return result, end_time - start_time  # Return the result and execution time


In [16]:
matrix_size = (100, 100)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

# print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

# print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 0.28499746322631836 секунд
Время на GPU: 0.04199838638305664 секунд


In [17]:
matrix_size = (250, 250)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

# print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

#print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 4.263211250305176 секунд
Время на GPU: 0.001001596450805664 секунд


In [18]:
matrix_size = (500, 500)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

#print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

#print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 35.20660090446472 секунд
Время на GPU: 0.22099924087524414 секунд


In [19]:
matrix_size = (1000, 1000)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

#print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

#print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 284.62263679504395 секунд
Время на GPU: 0.26085448265075684 секунд


In [20]:
matrix_size = (1500, 1500)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

#print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

#print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 967.6187109947205 секунд
Время на GPU: 0.15502214431762695 секунд


In [21]:
matrix_size = (2000, 2000)
matrix_a = np.random.rand(*matrix_size)
matrix_b = np.random.rand(*matrix_size)

result_cpu, time_cpu = matrix_multiply_cpu(matrix_a, matrix_b)
result_gpu, time_gpu = matrix_multiply_gpu(matrix_a, matrix_b)

#print(f"Результат на CPU:\n{result_cpu}")
print(f"Время на CPU: {time_cpu} секунд")

#print(f"Результат на GPU:\n{result_gpu}")
print(f"Время на GPU: {time_gpu} секунд")

Время на CPU: 2306.75697183609 секунд
Время на GPU: 0.22819042205810547 секунд
